In [ ]:
# Cell 1: インポートとセットアップ
import numpy as np
from scipy.fft import fft2, ifft2, fftfreq
import matplotlib.pyplot as plt
import torch
import time

# GPU使用の確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Cell 2: パラメータ設定（初学者解説: これらを調整して乱流強度を変えられます）
Nx, Ny = 128, 128  # 格子数（エクサスケールでは10^9以上ですが、ここは小規模）
Lx, Ly = 2*np.pi, 2*np.pi  # 領域サイズ
nu = 1e-3  # 動粘性係数（Re = 1/nu に比例）
dt = 1e-3  # 時間ステップ（安定性のためCFL条件を満たす）
n_steps = 1000  # シミュレーションステップ
plot_interval = 100  # プロット間隔

# 波数ベクトル（スペクトル法用）
kx = 2*np.pi/Lx * fftfreq(Nx, d=1.0) * Nx  # 正規化
ky = 2*np.pi/Ly * fftfreq(Ny, d=1.0) * Ny
Kx, Ky = np.meshgrid(kx, ky, indexing='ij')
K2 = Kx**2 + Ky**2  # Laplace演算子
K2[0,0] = 1e-10  # 分母ゼロ回避

# GPUへ転送
K2_t = torch.tensor(K2, dtype=torch.float32, device=device)
Kx_t = torch.tensor(Kx, dtype=torch.float32, device=device)
Ky_t = torch.tensor(Ky, dtype=torch.float32, device=device)

# Cell 3: 初期条件（ランダムな速度場で乱流を初期化）
def initial_vorticity(Nx, Ny):
    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)
    X, Y = np.meshgrid(x, y)
    omega = np.sin(2*np.pi*X/Lx) + 0.5*np.random.randn(Nx, Ny)  # 基本モード + ノイズ
    return omega

omega = initial_vorticity(Nx, Ny)
omega_t = torch.tensor(omega, dtype=torch.float32, device=device)

# Cell 4: スペクトル法の関数（正確な計算: 非線形項を物理空間で扱う）
def rhs_omega(omega_t):
    # 物理空間で速度を計算（Poissonから）
    psi_hat = -fft2(omega_t.cpu().numpy()) / K2  # ストリーム関数
    ux = np.real(ifft2(1j * Ky * psi_hat))  # u_x = dψ/dy
    uy = np.real(ifft2(-1j * Kx * psi_hat))  # u_y = -dψ/dx
    
    ux_t = torch.tensor(ux, device=device)
    uy_t = torch.tensor(uy, device=device)
    
    # 非線形項: (u · ∇)ω
    domega_dx = np.real(ifft2(1j * Kx * fft2(omega_t.cpu().numpy())))
    domega_dy = np.real(ifft2(1j * Ky * fft2(omega_t.cpu().numpy())))
    nonlinear = ux_t * torch.tensor(domega_dx, device=device) + uy_t * torch.tensor(domega_dy, device=device)
    
    # スペクトル空間に戻す
    nonlinear_hat = fft2(nonlinear.cpu().numpy())
    
    # 拡散項: ν ∇²ω
    diffusion_hat = -nu * K2 * fft2(omega_t.cpu().numpy())
    
    # 外力（オプション: ここではゼロ）
    forcing_hat = 0
    
    rhs_hat = -nonlinear_hat + diffusion_hat + forcing_hat
    return torch.tensor(np.real(ifft2(rhs_hat)), device=device)

# Cell 5: 時間積分（Runge-Kutta 4次: 正確性確保）
start_time = time.time()
for step in range(n_steps):
    k1 = dt * rhs_omega(omega_t)
    k2 = dt * rhs_omega(omega_t + 0.5*k1)
    k3 = dt * rhs_omega(omega_t + 0.5*k2)
    k4 = dt * rhs_omega(omega_t + k3)
    omega_t += (k1 + 2*k2 + 2*k3 + k4) / 6
    
    if step % plot_interval == 0:
        plt.imshow(omega_t.cpu().numpy(), cmap='jet')
        plt.title(f"Step {step}: Vorticity Field (Re ≈ {1/nu})")
        plt.colorbar()
        plt.show()

print(f"Simulation time: {time.time() - start_time:.2f} seconds")

# Cell 6: 結果分析（エネルギー спекtrumなど）
# 拡張として、Kolmogorov則を確認: E(k) ~ k^{-5/3}
omega_hat = fft2(omega_t.cpu().numpy())
energy = np.abs(omega_hat)**2 / (Nx*Ny)
k = np.sqrt(K2)
bins = np.arange(0, max(k.flatten()), 1)
energy_spectrum = np.histogram(k.flatten(), bins=bins, weights=energy.flatten())[0]
plt.loglog(bins[:-1], energy_spectrum)
plt.xlabel('Wavenumber k')
plt.ylabel('Energy E(k)')
plt.title('Energy Spectrum')
plt.show()